In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('Churn_UsersProfile.txt', delimiter = "|")
df.head()

## Filter for the two periods - do we need to normalise for the extra month on p1? - how?
### P2 during pandemic?

In [ ]:
period1 = (df.loc[df['Period'] == 'Sep-19 to Feb-20']
             .rename(columns={'Average_BusUsers_per_Day': 'Average_BusUsers_per_Day_p1'})
             .drop(columns = ['Period']))
period2 = (df.loc[df['Period'] == 'Sep-20 to Jan-21']
             .rename(columns={'Average_BusUsers_per_Day': 'Average_BusUsers_per_Day_p2'})
             .drop(columns = ['Period']))
period2.head(1)

 ## Join the two period tables to get a side by side view of Average Bus users per day per user profile

In [ ]:
df_merged = pd.merge(period1, period2, on=['Region_of_Origin', 'District_of_Origin', 'County_of_Origin',
                                           'GenderDescription', 'AgeClassDescription'])

In [ ]:
df_merged.head(5)

In [ ]:
df_merged['Churn'] = df_merged['Average_BusUsers_per_Day_p2'] - df_merged['Average_BusUsers_per_Day_p1']
df_merged['Churn_percent'] = ((df_merged['Average_BusUsers_per_Day_p2'] - df_merged['Average_BusUsers_per_Day_p1'])/ df_merged['Average_BusUsers_per_Day_p1'])*100

In [ ]:
df_merged.sort_values(by=['Churn_percent'], ascending = False).head(10)

## Analyse transport use changes for each age group

In [ ]:
df_age_comp = (df_merged.groupby(['AgeClassDescription'])
                        .agg({'Average_BusUsers_per_Day_p1': 'sum',
                              'Average_BusUsers_per_Day_p2': 'sum'}))

df_age_comp['Churn'] = df_age_comp['Average_BusUsers_per_Day_p2'] - df_age_comp['Average_BusUsers_per_Day_p1']
df_age_comp['Churn_percent'] = ((df_age_comp['Average_BusUsers_per_Day_p2'] - df_age_comp['Average_BusUsers_per_Day_p1'])/ df_age_comp['Average_BusUsers_per_Day_p1'])*100
df_age_comp.head(10)

In [ ]:
df_age_comp_churn = (df_merged.groupby(['AgeClassDescription'])
                              .agg({'Churn_percent': 'mean'}))
df_age_comp_churn.head(10)

## Analyse transport use changes per Region


In [ ]:
df_loc_comp = (df_merged.groupby(['Region_of_Origin', 'District_of_Origin','County_of_Origin'])
                        .agg({'Average_BusUsers_per_Day_p1': 'sum',
                              'Average_BusUsers_per_Day_p2': 'sum'}))

df_loc_comp['Churn'] = df_loc_comp['Average_BusUsers_per_Day_p2'] - df_loc_comp['Average_BusUsers_per_Day_p1']
df_loc_comp['Churn_percent'] = ((df_loc_comp['Average_BusUsers_per_Day_p2'] - df_loc_comp['Average_BusUsers_per_Day_p1'])/ df_loc_comp['Average_BusUsers_per_Day_p1'])*100
#df_loc_comp.head(100)

In [ ]:
df_loc_comp_churn = (df_merged.groupby(['Region_of_Origin', 'District_of_Origin','County_of_Origin'])
                        .agg({'Churn_percent': 'mean'}))
#df_loc_comp_churn.head(100)